*This jupyter notebook is part of Arizona State University's course CAS 570 (Introduction to Complex Systems Science) and was written by Bryan Daniels.  It was last updated November 6, 2023.*

*This notebook uses data gathered by Ying Wang and Robert E. Page, Jr. at Arizona State University.  The data can be accessed [here](https://figshare.com/articles/dataset/Data_Archive_for_Identifying_a_developmental_transition_in_honey_bees_using_gene_expression_data_/22696312).*

# Statistical analysis of honey bee gene expression data

In this notebook, we will practice using Principal Components Analysis to extract useful insights from a large-dimensional set of gene expression data.  We will see how a scientific question can be more easily approached when we visualize the data in a lower-dimensional space.

We will attempt to find a generative model that can output data that matches the statistics we measure in the data. In this case, the inferred model will take the form of a probability distribution of gene expression values, predicting which combinations of gene expression are more or less likely.  A typical challenge when inferring such models is selecting the best form of model:  Will a simple model suffice, or do we need to include more detail?  Here, we will use the "Bayesian information criterion" as a measure to decide which model is best.

This is part of a research project that I worked on together with Ying Wang, Rob Page, and Gro Amdam here at ASU, who are experts in honey bee physiology, behavior, and genetics.  Combining my expertise in physics and complex systems data analysis, this project is also a good example of the results of interdisciplinary collaboration.  Our writeup on the project can be found [here](https://doi.org/10.1371/journal.pcbi.1010704).

## Get set up and load the data

Let's load some useful basic packages and functions first:

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats
from gene_expression_example.landau import LandauDistributionPDF
plt.rcParams.update({'font.size': 18}) # increases font size on plots
#from helpers.prettyPlotting import scatter1D # custom 1D scatter plot
from pathlib import Path # to handle file paths across all operating systems

We will use the scikit learn function `sklearn.decomposition.PCA` to perform PCA.  The documentation is available [here](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html).

In [ ]:
from sklearn.decomposition import PCA

Now load the data:

In [ ]:
dataPath = Path('nanostring data with VG protein data.xlsx')
columnsToDrop = ['Gene','Unnamed: 2','Unnamed: 3','Sample ID','VG protein ']
expressionData = np.log(pd.read_excel(dataPath).drop(columns=columnsToDrop).set_index('Age'))

## What do these data represent?

These measurements were taken in honey bees at a precise time during their development (15 days old) when some bees are starting to leave the nest to forage for food.  Interestingly, some bees become foragers at a much younger age, while others stay in the nest much longer to take care of younger bees.  This transition is relatively sudden, with few bees switching back to in-nest activities once they start foraging.  There seem to be two separate "types" of bees related to which tasks they perform.  This is similar to how different cell types perform different tasks in your body.

Our question: As in cells in human development, are different bee types (those that perform distinct functions) related to which genes are expressed?

My collaborators chose genes to measure that were suspected to be related to the behavioral transition to foraging.  These data represent how strongly these genes are expressed in individual honey bees.  (Specifically, these are [measurements of the amount of RNA](https://en.wikipedia.org/wiki/RNA-Seq) present for each of the genes of interest.  We have taken the logarithm of the raw data to more easily capture wide variations in expression.)

Let's first look at the form of the data we have:

In [ ]:
age = 15 # days
expressionData.loc[age]

This is a `pandas` dataframe in which the columns represent the genes (90 of them) and the rows represent 16 individual bees whose gene expression was measured.

The default when printing a dataframe to the screen is to hide as many rows and columns as necessary to fit on a screen at once without a lot of scrolling.  To see the names of all the genes in the data, we can look at the `columns` attribute:

In [ ]:
expressionData.columns

# 1) Summarize

## The distribution of individual genes

In [ ]:
gene = 'vg'

# plot histogram
expressionData[gene].plot.hist(bins=20) #,density=True)
plt.xlabel('Expression of gene {}'.format(gene))
plt.ylabel('Number of bees');

In [ ]:
# fit a normal distribution
paramsNormal = stats.norm.fit(expressionData[gene])
print("Best-fit parameters: mean = {:1.5}, std. dev. = {:1.5}.".format(*paramsNormal))

In [ ]:
# plot histogram with overlaid best fit normal distribution
expressionData[gene].plot.hist(bins=20,density=True)
xs = np.linspace(8,15,100)
plt.plot(xs,[stats.norm.pdf(x,*paramsNormal) for x in xs],lw=5)
plt.xlabel('Expression of gene {}'.format(gene))
plt.ylabel('Probability density')
plt.title('Normal distribution');

## Try visualizing in 2D

Due to the large dimensionality of the dataset, it can be difficult to decide which aspects to focus on for thinking about our question about bee types.  Which genes are important?

One way to start is to visualize the data in lower dimensions by focusing on one or a few genes of interest at a time.  An easy way to do this using `pandas` is to use the `plot.scatter` function, which takes the names of two columns and constructs a scatter plot.  For example, we can visualize the expression of the genes *vg* and *ILP-2* in our 16 bees:

In [ ]:
expressionData.plot.scatter('vg','ILP-2');

In [ ]:
expressionData.plot.scatter('AKHR','proPO');

In [ ]:
expressionData.plot.scatter('ilp1','LOC102655054');

As I initially played around with these data, I happened to find that the pair of genes *vg* and *P110* made for an intriguing scatter plot, particularly when restricting to the oldest bees (age 15 days):

In [ ]:
expressionData.plot.scatter('vg','P110');

In [ ]:
expressionData.loc[15].plot.scatter('vg','P110');

# 2) Predict

In [ ]:
correlations = expressionData.corr()
# remove correlations with self
for gene in correlations.index:
    correlations.loc[(gene,gene)] = None

In [ ]:
# find the pair of genes with largest correlation
gene1 = correlations.max().idxmax()
gene2 = correlations.idxmax()[gene1]
print("The pair of genes with largest linear correlation is {} and {}.".format(gene1,gene2))
r,p = stats.pearsonr(expressionData[gene1],expressionData[gene2])
print("These have a correlation coefficient of {:1.5}, with p-value {:1.5}.".format(r,p))

In [ ]:
expressionData.plot.scatter('AGO1','Ftz-f1');

# 3) Reduce dimensionality

Instead of searching through many possible genes related to this transition, can we use dimensionality reduction to find one or a few dimensions that are particularly interesting?

Recall that Principal Components Analysis (PCA) is one way of picking out such dimensions: PCA chooses the dimensions with largest variance.  This could be useful for our question about bee types because, if gene expression varies with bee type, then we expect larger variance (and correlated variance) among the genes that define the distinct bee types.

The following code runs PCA on our expression data from the oldest bees (day 15), keeping only the 10 components with largest variance:

In [ ]:
age = 15 # days
pca_results = PCA(n_components=10).fit(expressionData.loc[age])

The results are stored as attributes of the `pca_results` object, which we explore below.  (If you are curious about what all is in there, recall how tab completion works in jupyter notebooks: you can type `pca_results.` followed by the tab key to see a list of the object's subparts.)

## How low-dimensional are the data?

As a first step for thinking about what PCA is doing, let's ask how much variance there is in the data along each of these first 10 components.  Specifically, we'll ask what proportion of the total variance lies along each principal component.  This is stored as `explained_variance_ratio_`:

In [ ]:
pca_results.explained_variance_ratio_

By construction, the first components have the largest variance (or "explain" the most variance, in the common lingo).

A common way of visualizing this is to plot the total variance included as a function of the number of principal components kept.  The following code computes this "cumulative sum" and plots it:

In [ ]:
var_explained_cumulative = pca_results.explained_variance_ratio_.cumsum()
plt.plot(np.arange(len(var_explained_cumulative))+1,var_explained_cumulative,'o:')
plt.xlabel('Number of principal components')
plt.ylabel('Proportion of\nvariance included')
plt.axis(xmin=1,ymax=1,ymin=0);

In [ ]:
var_explained_cumulative

## Interpreting the first principal component

For our question about bee types, it makes sense to focus on the first principal component (the one with largest variance): If the dissimilarity in bee behavior is connected strongly to gene expression, then we expect these large differences in behavior to correspond to large differences in gene expression.  We are looking for large variance!

The first principal component is stored in `pca_results` as `components_[0]` (I include the names of the genes here by creating a `pandas` series indexed by the names in `expressionData.columns`):

In [ ]:
component1 = pd.Series(pca_results.components_[0],
                       index = expressionData.columns)

Let's see what the first component looks like.  Recall that a principal component is defined in terms of weights given to each of the original dimensions (each of the original genes, in this case):

In [ ]:
component1

So the principal component is a list of length 90, with a weight for each gene (either positive or negative).

I typically find it useful to visualize things when possible.  Here's one way to visualize the principal component (I split into two plots for easier leigibility):

In [ ]:
plt.figure(figsize=(15,2))  # set up a large plot area
component1[:45].plot.bar(); # plot the weights of the first 45 genes

In [ ]:
plt.figure(figsize=(15,2))  # set up a large plot area
component1[45:].plot.bar(); # plot the weights of all genes past the first 45

How to interpret these results?  Most genes don't contribute much to the principal component (they have small weights), and a few contribute a lot.  One way to find the genes that contribute most is to sort by the absolute value of their weight:

In [ ]:
abs(component1).sort_values()

So *hex 110* has the largest contribution, followed by *Hex70a*, and so on.

## Reducing data to a single dimension

Of course, the point of dimensionality reduction is that we can look at the data using these reduced coordinates.  In the extreme case, instead of the full dimensionality of the dataset, we can characterize each sample (each bee) by a *single* number.  This number is the "linear projection" of the full dimensional data onto the principal component—that is, we weight the gene expression of each bee by multiplying by the weights of the first principal component, then add them up to get a single value.

This projection, also called a "dot product", is accomplished by `np.dot`:

In [ ]:
data_along_component1 = np.dot(expressionData.loc[age],component1)

Projected along the first principal component, our dataset is reduced to 16 single numbers, one for each bee:

In [ ]:
data_along_component1

We might make a histogram to visualize the data:

In [ ]:
plt.hist(data_along_component1,bins=10)
plt.xlabel('Distance along first component')
plt.ylabel('Number of bees')

# Separate bees into potential groups

We might separate bees into groups by setting a threshold along the first principal component.

Insert your threshold into the following code, which then splits the bees into two groups and assigns them colors based on which group they are in.

In [ ]:
threshold = -8 
beesA = np.where(data_along_component1 > threshold)[0]
beesB = np.where(data_along_component1 < threshold)[0]

# make list of colors based on the group
colors = []
for i in range(16):
    if i in beesA: 
        colors.append('crimson')
    else: 
        colors.append('cornflowerblue')

Here's an example using the colors in a scatter plot (where red dots correspond to bees in group A, and blue to group B):

In [ ]:
# largest contributors to component 1
expressionData.loc[age].plot.scatter('hex 110','Hex70a',
                            c=colors,s=100);

In [ ]:
expressionData.loc[age].plot.scatter('vg','P110',
                            c=colors,s=100);

# 4) Simulate

Now we will aim for a generative model.  A generative model acts as a hypothesis that produces data similar to what we observed in the actual system.

In [ ]:
expressionDataDay1 = expressionData.loc[1]
expressionDataDay15 = expressionData.loc[15]

pcaProjectionsDay1 = PCA(n_components=1).fit_transform(expressionDataDay1)
pcaProjectionsDay15 = PCA(n_components=1).fit_transform(expressionDataDay15)

To create a generative model, can we find a probability distribution that gives a reasonable approximation of the data along the principal component?

First try a normal distribution:

In [ ]:
paramsNormalDay15 = stats.norm.fit(pcaProjectionsDay15)

In [ ]:
# plot the best-fit normal distribution
plt.hist(pcaProjectionsDay15,bins=10,density=True)
xs = np.linspace(-15,15,100)
plt.plot(xs,[stats.norm.pdf(x,*paramsNormalDay15) for x in xs],lw=5)
plt.xlabel('Distance along first component')
plt.ylabel('Probability density')
plt.title('Day 15, Unimodal distribution fit');

The function `nnlf` returns the negative log-likelihood of the data given the model, so `-nnlf` is the standard log-likelihood:

In [ ]:
normalLogLDay15 = -stats.norm.nnlf(paramsNormalDay15,pcaProjectionsDay15)[0]
print("The log-likelihood for the Normal distribution model on Day 15 data is {}.".format(normalLogLDay15))

In a particular mathematical sense that I won't describe in detail here, the simplest distribution with *two* peaks (bimodal) is what I will call the "Landau" distribution.  This distribution corresponds to a very simplistic model of phase transitions in statistical physics. Here's the form of the distribution in case you are interested:

$$
p(x) = \frac{1}{Z} \exp \left( -\frac{c}{2} (x-\mu)^2 - \frac{d}{4} (x-\mu)^4 \right),
$$

where $Z$ is a normalization constant.  

The Landau distribution has three parameters ($\mu$, $c$, and $d$) compared to the Gaussian model's two parameters ($\mu$ and $\sigma$).  Note that the Gaussian is a special case of the Landau distribution: setting $d=0$ gives the usual form of a Gaussian.  So this model can have a single peak or two peaks depending on the values of the parameters.

The Landau distribution is not standard enough to be implemented in the packages we use here, so I've done the fitting of parameters for you.  We load these parameters here:

In [ ]:
landauFitParameters = pd.read_csv(Path('gene_expression_example/landau_fit_parameters.csv'),index_col='age')

In [ ]:
# plot the best-fit Landau distribution
age = 15 # age of bees in days
params = landauFitParameters.loc[age]
plt.hist(pcaProjectionsDay15,bins=10,density=True)
xs = np.linspace(-15,15,100)
plt.plot(xs,LandauDistributionPDF(xs,params['mu'],params['c'],params['d']),lw=5)
plt.xlabel('Distance along first component')
plt.ylabel('Probability density')
plt.title('Day 15, Bimodal distribution fit');

This looks like a much better fit!  Still, it's not exactly right—but then again, we only have 16 datapoints, so we don't expect the fit to be exactly right.  How do we decide if it's better enough compared to the unimodal distribution?  This is a job for statistical model selection!

First, let's look at the log-likelihood of the data given the Landau model.  I also have this pre-saved:

In [ ]:
landauLogLDay15 = landauFitParameters.loc[15]['log-likelihood']
print("The log-likelihood for the Landau model on Day 15 data is {}.".format(landauLogLDay15))

This is a better fit according to the log-likelihood.

But notice that the unimodal distribution is a special case of the bimodal distribution.
That is, after we have added new parameters, we could still produce the same unimodal behavior as before.

In this case, it is not possible to be forced to have a worse fit with the more complicated model — you could always just use the setting of the new parameter that gives the same answer as the simpler model.  So the new, larger model will *always* fit better, no matter the data.  (Well, I suppose it could also fit the same—it's just not possible for the fit to get worse.)

This logic means that our better fit with the more complicated model is not so impressive.  In other language we have encountered in this class, it's possible that the new, more complicated model is simply *overfitting*: fitting noise in the data.

To determine if the extra parameter is "worth the trouble", we will compute the Bayesian information criterion (BIC).  The BIC includes a penalty on more complicated models, essentially setting a bar for *how much* better a model with extra parameters must fit the data to be statistically favored.

The BIC is defined as
$$
\textrm{BIC} = L - \frac{1}{2} N_{params} \log(N_{datapoints}),
$$
where $L$ is the log-likelihood, $N_{params}$ is the number of free parameters in the model, and $N_{datapoints}$ is the number of datapoints to which the model was fit.

The following code computes BIC for the Normal and Landau models.

In [ ]:
# compute bic for normal distribution model
numParamsNormal = 2
numDatapoints = 16
normalBICDay15 = normalLogLDay15 - numParamsNormal/2 * np.log(numDatapoints)

# compute bic for landau distribution model
numParamsLandau = 3
numDatapoints = 16
landauBICDay15 = landauLogLDay15 - numParamsLandau/2 * np.log(numDatapoints)

In [ ]:
print("Day 15:")
print("    The Gaussian model has BIC = {} and the Landau model has BIC = {}.".format(
    normalBICDay15,landauBICDay15))
print("    The Landau model has larger BIC by a difference of {}.".format(
    landauBICDay15-normalBICDay15))

In [ ]:
# plot the best-fit Landau distribution
age = 1 # age of bees in days
params = landauFitParameters.loc[age]
plt.hist(pcaProjectionsDay1,bins=10,density=True)
xs = np.linspace(-15,15,100)
plt.plot(xs,LandauDistributionPDF(xs,params['mu'],params['c'],params['d']),lw=5)
plt.xlabel('Distance along first component')
plt.ylabel('Probability density')
plt.title('Day 1, Bimodal distribution fit');